# Tuning hyperparameters for domain

This notebook explores the hyperparameters for generating a domain. The domain as assumed to be a simply non-hierarchical structure where all kc's are independent.

## Setup

In [ ]:
import sys
sys.path.append("../lib")

In [27]:
import math
import random
import uuid
import os
import copy
import itertools
from collections.abc import Iterable
import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import Bounds
from scipy.optimize import minimize
from scipy import optimize
from scipy.stats import pearsonr

from sklearn.cluster import DBSCAN
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm



In [3]:
import logging

#logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(level=logging.INFO)
#logging.basicConfig(level=logging.WARNING)

logger = logging.getLogger("main")

In [4]:
#logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger().setLevel(logging.INFO)
#logging.getLogger().setLevel(logging.WARNING)
logger.debug("Test debug")
logger.info("Test info")
logger.warning("Test warning")

INFO:main:Test info


In [5]:
# from tutor.domain import Domain
from tutor.curriculum_factory import CurriculumFactory
from tutor.simple_curriculum import SimpleCurriculum
from tutor.tutor import SimpleTutor
from tutor.action import Attempt, HintRequest

In [6]:
from learner.selfeff_learner import SelfEfficacyLearner
from learner.modular_learner import ModularLearner
from learner.cognition import *
from learner.decider import *

In [7]:
from simulate.modlearner_simulation import ModLearnerSimulation
from simulate.simulation import *

In [8]:
from analytics.student_stats import *
from analytics.cae import *
from analytics.featurization import *
from analytics.batch import *

In [9]:
from log_db import mongo
from log_db.curriculum_mapper import DB_Curriculum_Mapper
from log_db.learner_mapper import DBLearnerMapper

In [10]:
from CanonicalAutocorrelationAnalysis.model.caa import CAAComputation
from CanonicalAutocorrelationAnalysis.model.caaObject import *
from CanonicalAutocorrelationAnalysis.model.utils import l1Norm, l2Norm, r2Compute

In [11]:
# Get path to current project directory
cwd = os.path.abspath(".")
base_dir = os.path.abspath(os.path.join(cwd, os.pardir))
logger.debug("Base directory for the project:\n%s" % base_dir)

In [12]:
# Setup connection to database
data_out = "sim-%s" % str(uuid.uuid4())
data_path = os.path.join(base_dir,"test", "data", data_out)
logger.info("Writing simulation results to directory: %s" % data_path)
db_name = "motivsim"
db_params  = mongo.get_db_params(db_name)
db_params['port'] = '27018'
logger.info("got db params: %s" % str(db_params))
db_util = mongo.Data_Utility(data_path, db_params)
db = db_util.db

INFO:analytics.batch:Writing simulation results to directory: /rdata/Sandbox/MotivSim/test/data/sim-81ca77f6-b092-4e8a-9e24-5ad820fd7217
INFO:analytics.batch:got db params: {'settingId': 'motivsim', 'url': 'localhost', 'port': '27018', 'name': 'motivsim', 'user': '', 'pswd': ''}


In [13]:
# Test db connection
db_util.peak()

INFO:log_db.mongo:collection name, steps, has 0 documents
INFO:log_db.mongo:collection name, problems, has 0 documents
INFO:log_db.mongo:collection name, curriculums, has 0 documents
INFO:log_db.mongo:collection name, kcs, has 0 documents
INFO:log_db.mongo:collection name, students, has 0 documents
INFO:log_db.mongo:collection name, units, has 0 documents
INFO:log_db.mongo:collection name, simbatches, has 0 documents
INFO:log_db.mongo:collection name, tutor_events, has 0 documents
INFO:log_db.mongo:collection name, domains, has 0 documents
INFO:log_db.mongo:collection name, sessions, has 0 documents
INFO:log_db.mongo:collection name, decisions, has 0 documents
INFO:log_db.mongo:collection name, actions, has 0 documents
INFO:log_db.mongo:collection name, sections, has 0 documents
INFO:log_db.mongo:collection name, finalsimstudents, has 0 documents


In [14]:
clear_db = True
if clear_db:
    logger.info("Clearing database before starting new simulation")
    db_util.clear_db()
else:
    logger.info("Skipping Clearing database")

INFO:analytics.batch:Clearing database before starting new simulation


## Multi-simulation run

### Simulation Run Parameters

In [11]:
num_sims=2
stu_per_sim = 2

### Simualtion Runs

In [36]:
def gen_curriculum(domain,
                   num_units=2,
                   mean_sections=4,
                   stdev_sections=2,
                   mean_unit_kcs=5,
                   stdev_unit_kcs=5,
                   section_kcs_lambda=2,
                   mean_steps=3,
                   stdev_steps=1,
                   mean_prob_kcs=2,
                   stdev_prob_kcs=1,
                   num_practice=100,
                   
                  ):# Generating the Curriculum and domain together
#    domain = Domain()
    curric = CogTutorCurriculum(domain)
    curric.generate(num_units=num_units,
                   mean_sections=mean_sections,
                   stdev_sections=stdev_sections,
                   mean_unit_kcs=mean_unit_kcs,
                   stdev_unit_kcs=stdev_unit_kcs,
                   section_kcs_lambda=section_kcs_lambda,
                   mean_steps=mean_steps,
                   stdev_steps=stdev_steps,
                   mean_prob_kcs=mean_prob_kcs,
                   stdev_prob_kcs=stdev_prob_kcs,
                   num_practice=num_practice
                   )
    
    # Insert domain to db
    #db.domains.insert_one(domain.to_dict())
    #db.kcs.insert_many([kc.__dict__ for kc in domain.kcs])

    # Insert Curriculum to db
    #curric_util = DB_Curriculum_Mapper(db_params)
    #curric_util.write_to_db(curric)
    
    return curric

In [13]:
def gen_students(domain, num_students=2):
    students = [DomainTuner(domain) for i in range(num_students)]
    logger.info("Sample student:\n%s" % str(students[0]))
    logger.info("Inserting %i students to db" % len(students))
    result = db.students.insert_many([stu.to_dict() for stu in students])
    logger.info("Db insert success: %s" % result.acknowledged)
    return students

In [14]:
def simulate_students(domain, curric, students):    
    for i, stu in enumerate(students):
        logger.info("Simulating student #%i" % i)
        sim = SelfEffSimulation(domain, curric, stu)
        sim.run()

In [15]:
def calc_sim_stats(curric, students):

    stu_ids = [stu._id for stu in students]
    tx = pd.DataFrame(db.tutor_events.find({'type': "Tutor Input", 'stu_id': {"$in": stu_ids}}))
    logger.info("Learner Transactions: %s" % str(tx.shape))
    # Add kc field that reduces list of kcs to 1 kc
    tx['kc'] = tx.apply(lambda x: x['kcs'][0]['_id'], axis=1)
    
    # Aggregate Learner transaction stats

    # Total Transaction counts
    stu_stats = tx.groupby('stu_id').agg({'_id': 'count', 
                                          'duration': np.sum,
                                         })
    stu_stats.rename(columns={'_id': "Total Tx",
                              'duration': 'Total Time'}, 
                             inplace = True)
    stu_stats['Total Time(hours)'] = stu_stats['Total Time'].apply(lambda x: x / 3600)
    #logger.info("Number of students: %i" % stu_stats.shape[0])
    #logger.info(stu_stats["Total Tx"].describe())

    # Total of each outcome
    d = tx.groupby(['stu_id','outcome'])['_id'].count().reset_index().pivot(index='stu_id', columns='outcome', values='_id')
    # Proporation of each outcome
    if len(d.columns) > 1:
        d['Total'] = d.sum(axis=1)
    else:
        d['Total'] = d.iloc[:,0]

    for col in d.columns:
        if col != 'Total':
            d['Pct %s' % col] = d[col] / d['Total']
    stu_stats = pd.concat([stu_stats, d], axis=1)
    
    # Calculate attempts per ste
    stu_step_stats = tx.groupby(['stu_id', 'step_id'])['_id'].count().reset_index()
    stu_step_stats.rename(columns={'_id': 'step attempts'}, inplace=True)
    step_stats = stu_step_stats.groupby('stu_id').describe()
    
    # Calculate opportunities per kc per student
    stu_kc_stats = tx[['stu_id', 'kc', 'step_id']].drop_duplicates().groupby(['stu_id', 'kc']).count().reset_index()
    stu_kc_stats.rename(columns={'step_id': 'kc opportunities'}, inplace=True)
    kc_stats = stu_kc_stats.groupby('kc').describe()

    
    # consolidate distributional stats of resulting data

    stats = {}
    stats['Step attempts mean'] = step_stats[('step attempts', 'mean')].mean()
    stats['Step attempts std'] = step_stats[('step attempts', 'std')].mean()
    accuracy_dist = stu_stats['Pct Correct'].describe()
    stats['Mean Pct Correct'] = accuracy_dist['mean']
    stats['Std Pct Correct'] = accuracy_dist['std']
    stats['KC opportunity mean'] = kc_stats[('kc opportunities', 'mean')].mean()
    stats['KC opportunity std'] = kc_stats[('kc opportunities', 'std')].mean()
    return stats

In [16]:
def run_sim(num_stu, hyperparams=None):
    domain = Domain()
    # Set domain hyperparams
    if hyperparams is not None:
        domain.set_kc_hyperparams(**hyperparams)
    logger.info("*** domain has %i kcs before curric *****" % len(domain.kcs) )
    curric = gen_curriculum(domain)
    logger.info("*** domain has %i kcs *****" % len(domain.kcs) )
    students = gen_students(domain, num_stu)
    simulate_students(domain, curric, students)
    stats = calc_sim_stats(curric, students)

    return stats

### Tuning Objective

Tuning on distribution of opportunities per kc, attempts per step, and student accuracy


In [17]:
target = {
    'Step attempts mean': 1,
    'Step attempts std': 0.4,
    'Mean Pct Correct': 0.8,
    'Std Pct Correct': 0.1,
    'KC opportunity mean': 7,
    'KC opportunity std': 3
}

def target_obj(pred, target=target):
    d = pd.DataFrame([pred, target])
    err = math.sqrt(np.sum(d.apply(lambda x: (x[1] - x[0])**2, axis=0)))
    return err


In [18]:

def eval_sim_params(params, num_sims, stu_per_sim):
    start = dt.datetime.now()
    logger.error("running eval sim %i times with params: %s" % (num_sims, str(params)))
    db_util = mongo.Data_Utility(data_path, db_params)
    param_dict = {
            'm_l0': params[0],
            'sd_l0':params[1],
            'm_t':params[2],
            'sd_t':params[3],
            'm_s':params[4],
            'sd_s':params[5],
            'm_g':params[6],
            'sd_g':params[7]
    }
    errs = []
    for i in range(num_sims):
        result = run_sim(stu_per_sim, param_dict)
        errs.append(target_obj(result))
        #param_dict['error'] = err
    db_util.clear_db()
    end = dt.datetime.now()
    err = np.mean(errs)
    stdev_err = np.std(errs)
    logger.error("Run took %f seconds with mean err: %f\t stdev err: %f" % (((end - start).total_seconds(), err, stdev_err)))

    return err

    

### Run Tuning simulation runs

In [17]:
# Test specific case
#params = [0.7,  0.3,  0.8,  0.05, 0.3,  0.1,  1.0,  0.01]
#params = [0.9,  0.3,  0.1,  0.01, 0.01, 0.01, 0.2,  0.01]
params = [0.45 , 0.155, 0.45 , 0.03 , 0.155, 0.055, 0.6  , 0.105]
reps = 10  # Set to zero so it won't accidntally execute

for i in range(reps):
    start = dt.datetime.now()
    result = eval_sim_params(params, num_sims, stu_per_sim)
    end = dt.datetime.now()
    time = (end - start).total_seconds()    
print("Done")

ERROR:main:running eval sim 2 times with params: [0.45, 0.155, 0.45, 0.03, 0.155, 0.055, 0.6, 0.105]
ERROR:main:Run took 0.842972 seconds with mean err: 2.977656	 stdev err: 0.433265
ERROR:main:running eval sim 2 times with params: [0.45, 0.155, 0.45, 0.03, 0.155, 0.055, 0.6, 0.105]
ERROR:main:Run took 0.899673 seconds with mean err: 1.611873	 stdev err: 0.388029
ERROR:main:running eval sim 2 times with params: [0.45, 0.155, 0.45, 0.03, 0.155, 0.055, 0.6, 0.105]
ERROR:main:Run took 0.343649 seconds with mean err: 3.004740	 stdev err: 0.369121
ERROR:main:running eval sim 2 times with params: [0.45, 0.155, 0.45, 0.03, 0.155, 0.055, 0.6, 0.105]
ERROR:main:Run took 0.687735 seconds with mean err: 2.968252	 stdev err: 0.137057
ERROR:main:running eval sim 2 times with params: [0.45, 0.155, 0.45, 0.03, 0.155, 0.055, 0.6, 0.105]
ERROR:main:Run took 0.924283 seconds with mean err: 1.676432	 stdev err: 0.070698
ERROR:main:running eval sim 2 times with params: [0.45, 0.155, 0.45, 0.03, 0.155, 0.0

Done


In [18]:
reps = 0
times = []
for i in range(reps):
    params = [
        random.uniform(bounds[0][0], bounds[0][1]),
        random.uniform(bounds[1][0], bounds[1][1]),
        random.uniform(bounds[2][0], bounds[2][1]),
        random.uniform(bounds[3][0], bounds[3][1]),
        random.uniform(bounds[4][0], bounds[4][1]),
        random.uniform(bounds[5][0], bounds[5][1]),
        random.uniform(bounds[6][0], bounds[6][1]),
        random.uniform(bounds[7][0], bounds[7][1])
    ]
    start = dt.datetime.now()
    result = eval_sim_params(params, num_sims, stu_per_sim)
    end = dt.datetime.now()
    times.append((end - start).total_seconds())
    #print("Run took %f seconds" % times[-1])
    #print(result)
print("average run time: %f seconds" % np.mean(times))

average run time: nan seconds


/rdata/Sandbox/MotivSim/notebooks/jvenv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/rdata/Sandbox/MotivSim/notebooks/jvenv/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [19]:
#bounds = [(0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
#          (0.7, 0.3, 0.8, 0.05, 0.3, 0.1, 1.0, 0.2)]
bounds = [(0.2,0.7), (0.01,0.3), (0.1,0.8), (0.01,0.05),
        (0.01,0.3), (0.01,0.1), (0.2,1.0),(0.01,0.2)]
#init_params = list(params.values())
#print("Initial parameters: %s" % str(init_params))
print("Bounds: %s" % str(bounds))
start = dt.datetime.now()
min_result = optimize.shgo(eval_sim_params, bounds, args=(num_sims, stu_per_sim))
end = dt.datetime.now()
print("operation took: %s" % str((end - start)))
print("Minimize result: %s" % str(min_result))

ERROR:main:running eval sim 2 times with params: [0.2  0.01 0.1  0.01 0.01 0.01 0.2  0.01]


Bounds: [(0.2, 0.7), (0.01, 0.3), (0.1, 0.8), (0.01, 0.05), (0.01, 0.3), (0.01, 0.1), (0.2, 1.0), (0.01, 0.2)]


ERROR:main:Run took 1.245914 seconds with mean err: 12.305095	 stdev err: 2.896014
ERROR:main:running eval sim 2 times with params: [0.7  0.3  0.8  0.05 0.3  0.1  1.   0.2 ]
ERROR:main:Run took 0.717358 seconds with mean err: 4.953019	 stdev err: 0.043291
ERROR:main:running eval sim 2 times with params: [0.7  0.01 0.1  0.01 0.01 0.01 0.2  0.01]
ERROR:main:Run took 3.211063 seconds with mean err: 6.577959	 stdev err: 1.331609
ERROR:main:running eval sim 2 times with params: [0.7  0.3  0.1  0.01 0.01 0.01 0.2  0.01]
ERROR:main:Run took 1.332894 seconds with mean err: 6.366624	 stdev err: 3.783213
ERROR:main:running eval sim 2 times with params: [0.7  0.3  0.8  0.01 0.01 0.01 0.2  0.01]
ERROR:main:Run took 0.388587 seconds with mean err: 4.359818	 stdev err: 0.111436
ERROR:main:running eval sim 2 times with params: [0.7  0.3  0.8  0.05 0.01 0.01 0.2  0.01]
ERROR:main:Run took 1.037300 seconds with mean err: 4.296689	 stdev err: 0.157219
ERROR:main:running eval sim 2 times with params: [0.

operation took: 0:13:45.968088
Minimize result:      fun: 11.917028956240312
    funl: array([11.91702896])
 message: 'Optimization terminated successfully.'
    nfev: 266
     nit: 2
   nlfev: 9
   nlhev: 0
   nljev: 1
 success: True
       x: array([0.7 , 0.3 , 0.1 , 0.05, 0.3 , 0.01, 0.2 , 0.01])
      xl: array([[0.7 , 0.3 , 0.1 , 0.05, 0.3 , 0.01, 0.2 , 0.01]])


#### Optimial Result:


m_l0: 0.45 <br>
sd_l0: 0.155  <br>
m_t: 0.45  <br>
sd_t: 0.03  <br>
m_s: 0.155 <br>
sd_s: 0.055 <br>
m_g: 0.6 <br>
sd_g: 0.105 <br>

In [ ]:

bnds = Bounds([0.2, 0.01, 0.1, 0.01, 0.01, 0.01, 0.2, 0.01],
               [0.7, 0.3, 0.8, 0.05, 0.3, 0.1, 1.0, 0.2])

In [ ]:
# Local minima near global minima
init_params = min_result.x
print("Initial parameters: %s" % str(init_params))
print("Bounds: %s" % str(bnds))
start = dt.datetime.now()
# Commented out for now
min_result = minimize(eval_sim_params, init_params, 
                      args=(num_sims, stu_per_sim),
                      method='powell', bounds=bnds)
print("operation took: %s" % str((end - start)))
print("Minimize result: %s" % str(min_result))

 <br>### EDA of Simulation Runs

# Tuning Domain Model for Continuous KC

In [15]:
clear_db = True
if clear_db:
    logger.info("Clearing database before starting new simulation")
    db_util.clear_db()
else:
    logger.info("Skipping Clearing database")

INFO:analytics.batch:Clearing database before starting new simulation


## Helper Functions

In [16]:
def gen_curriculum(db, db_params, domain_params):
        logger.debug("Generating new curriculum")
        
        curric_params = {'num_units': 2,
                         'mean_sections': 4,
                         'stdev_sections': 2,
                         'mean_unit_kcs': 22,
                         'stdev_unit_kcs': 23,
                         'section_kcs_lambda': 6,
                         'mean_steps': 10,
                         'stdev_steps': 4,
                         'mean_prob_kcs': 6,
                         'stdev_prob_kcs': 3,
                         'num_practice': 100
                        }

        domain, curric = CurriculumFactory.gen_curriculum(domain_params, curric_params)

        return domain, curric


In [17]:
def get_cog_params():
    ability = -2
    while (ability < -1) or (ability > 1):
        ability = np.random.normal(0, 0.6)
    return {"ability": ability}

def get_dec_params():
    return {}

def gen_students(num_students, domain, curric, 
                 cog_mod, cog_params, dec_mod, dec_params):
    stus = []
    for i in range(num_students):
        cp = cog_params()
        cog = cog_mod(domain, **cp)
        dp = dec_params()
        dec = dec_mod(**dp)
        decider = DiligentDecider(dec)
        stu = ModularLearner(domain, cog, decider)
        stus.append(stu)
        
    return stus

def simulate_students(curric, students, batch):    
    
    env = simpy.Environment()

    mastery_thres = 0.9
    m_ses_len = 40
    sd_ses_len = 8
    max_ses_len = 60
    sim_start = dt.datetime.now()

    mod = round(len(students) / 10)
    for i, stu in enumerate(students):
#         if i % mod == 0:
#             logger.info("Simulating student #%i" % i)
        # Create associated tutor
        tutor = SimpleTutor(curric, stu._id, mastery_thres)

        # Initialize simulation processes
        num_sessions = 20
        sim = SingleStudentSim(db, env, sim_start, stu, tutor,
                               num_sessions, m_ses_len, sd_ses_len, max_ses_len)
        batch.add_sim(sim)

        env.process(sim.run())

    env.run()

    return batch, students

In [18]:
def calc_sim_stats(curric, students):
    logger.info("Calculating stats for students")
    stu_ids = [stu._id for stu in students]
    tx = pd.DataFrame(db.tutor_events.find({'type': "TutorInput", 'stu_id': {"$in": stu_ids}}))
    logger.info("Learner Transactions: %s" % str(tx.shape))
    # Add kc field that reduces list of kcs to 1 kc
    tx['kc'] = tx.apply(lambda x: x['kcs'][0]['_id'], axis=1)
    
    # Aggregate Learner transaction stats

    # Total Transaction counts
    stu_stats = tx.groupby('stu_id').agg({'_id': 'count', 
                                          'duration': np.sum,
                                         })
    stu_stats.rename(columns={'_id': "Total Tx",
                              'duration': 'Total Time'}, 
                             inplace = True)
    stu_stats['Total Time(hours)'] = stu_stats['Total Time'].apply(lambda x: x / 3600)
    #logger.info("Number of students: %i" % stu_stats.shape[0])
    #logger.info(stu_stats["Total Tx"].describe())

    # Total of each outcome
    d = tx.groupby(['stu_id','outcome'])['_id'].count().reset_index().pivot(index='stu_id', columns='outcome', values='_id')
    # Proporation of each outcome
    if len(d.columns) > 1:
        d['Total'] = d.sum(axis=1)
    else:
        d['Total'] = d.iloc[:,0]

    for col in d.columns:
        if col != 'Total':
            d['Pct %s' % col] = d[col] / d['Total']
    stu_stats = pd.concat([stu_stats, d], axis=1)
    
    # Calculate attempts per ste
    stu_step_stats = tx.groupby(['stu_id', 'step_id'])['_id'].count().reset_index()
    stu_step_stats.rename(columns={'_id': 'step attempts'}, inplace=True)
    step_stats = stu_step_stats.groupby('stu_id').describe()
    
    # Calculate opportunities per kc per student
    stu_kc_stats = tx[['stu_id', 'kc', 'step_id']].drop_duplicates().groupby(['stu_id', 'kc']).count().reset_index()
    stu_kc_stats.rename(columns={'step_id': 'kc opportunities'}, inplace=True)
    kc_stats = stu_kc_stats.groupby('kc').describe()

    
    # consolidate distributional stats of resulting data

    stats = {}
    stats['Step attempts mean'] = step_stats[('step attempts', 'mean')].mean()
    stats['Step attempts std'] = step_stats[('step attempts', 'std')].mean()
    accuracy_dist = stu_stats['Pct Correct'].describe()
    stats['Mean Pct Correct'] = accuracy_dist['mean']
    stats['Std Pct Correct'] = accuracy_dist['std']
    stats['KC opportunity mean'] = kc_stats[('kc opportunities', 'mean')].mean()
    stats['KC opportunity std'] = kc_stats[('kc opportunities', 'std')].mean()
    return stats

In [19]:
def run_sim(num_stu, hyper_params, db, db_params):
    
    domain, curric = gen_curriculum(db, db_params, hyper_params)

    students = gen_students(num_stu, domain, curric, 
                            cog_mod, get_cog_params, 
                            dec_mod, get_dec_params)
    batch = SimulationBatch("Tune DB Student Sim")

    simulate_students(curric, students, batch)    
    stats = calc_sim_stats(curric, students)

    return stats

### Tuning Objective

Tuning on distribution of opportunities per kc, attempts per step, and student accuracy


In [20]:
target = {
    'Step attempts mean': 1,
    'Step attempts std': 0.4,
    'Mean Pct Correct': 0.8,
    'Std Pct Correct': 0.1,
    'KC opportunity mean': 7,
    'KC opportunity std': 3
}

def target_obj(pred, target=target):
    d = pd.DataFrame([pred, target])
    err = math.sqrt(np.sum(d.apply(lambda x: (x[1] - x[0])**2, axis=0)))
    return err


In [21]:

def eval_sim_params(params, num_sims, stu_per_sim, db, db_params):
    start = dt.datetime.now()
    logger.error("running eval sim %i times with params: %s" % (num_sims, str(params)))
    param_keys = ['m_l0','sd_l0','m_l0_sd','sd_l0_sd','m_t','sd_t','m_s','sd_s','m_g','sd_g']
    param_dict = {d[0]: d[1] for d in zip(param_keys, params)}
    errs = []
    for i in range(num_sims):
        result = run_sim(stu_per_sim, param_dict, db, db_params)
        errs.append(target_obj(result))
        #param_dict['error'] = err
    db_util.clear_db()
    end = dt.datetime.now()
    err = np.mean(errs)
    stdev_err = np.std(errs)
    logger.info(f"Error:\t{result}")
    logger.error("Run took %f seconds with mean err: %f\t stdev err: %f" % (((end - start).total_seconds(), err, stdev_err)))

    return err

    

### Run Tuning simulation runs

In [22]:
num_sims=2
stu_per_sim = 20

cog_mod = BiasSkillCognition
dec_mod = DomainTunerDecider

In [23]:
# Test specific case

params = [0.45 , 0.155, 
          0.1, 0.03, 
          0.1 , 0.05 , 
          0.155, 0.055, 
          0.6  , 0.205]

params = [0.7,  0.3,  0.2,  0.2,  0.25, 0.05, 0.3,  0.01, 0.2,  0.15]
#params = [0.69975586, 0.0101416 , 0.0101416 , 0.19990723, 0.1       ,
#       0.04998047, 0.3       , 0.01004395, 0.99960938, 0.01009277]
# Zero this out so the cell won't accidentally run
reps = 1

for i in range(reps):
    start = dt.datetime.now()
    result = eval_sim_params(params, num_sims, stu_per_sim, db, db_params)
    end = dt.datetime.now()
    time = (end - start).total_seconds()    
print("Done")

ERROR:analytics.batch:running eval sim 2 times with params: [0.7, 0.3, 0.2, 0.2, 0.25, 0.05, 0.3, 0.01, 0.2, 0.15]
INFO:tutor.cogtutor_curriculum:Generated 2 units with with a total of 51 kcs
INFO:analytics.batch:Calculating stats for students
INFO:analytics.batch:Learner Transactions: (5902, 18)
INFO:tutor.cogtutor_curriculum:Generated 2 units with with a total of 110 kcs
INFO:analytics.batch:Calculating stats for students
INFO:analytics.batch:Learner Transactions: (9932, 18)
INFO:analytics.batch:Error:	{'Step attempts mean': 4.235401586880833, 'Step attempts std': 0.9072740826164307, 'Mean Pct Correct': 0.6112614935077165, 'Std Pct Correct': 0.06145833602580424, 'KC opportunity mean': 5.782407885454104, 'KC opportunity std': 2.803970969411856}
ERROR:analytics.batch:Run took 16.023349 seconds with mean err: 3.333126	 stdev err: 0.171616


Done


In [25]:
#bounds = [(0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
#          (0.7, 0.3, 0.8, 0.05, 0.3, 0.1, 1.0, 0.2)]
bounds = [(0.2,0.7), (0.05,0.3), 
          (0.05,0.2), (0.01,0.2),
          (0.05,0.25), (0.01,0.05),
          (0.05,0.3), (0.01,0.1), 
          (0.2,0.5), (0.02,0.15)]
#init_params = list(params.values())
#print("Initial parameters: %s" % str(init_params))
print("Bounds: %s" % str(bounds))
start = dt.datetime.now()
min_result = optimize.shgo(eval_sim_params, bounds, args=(num_sims, stu_per_sim, db, db_params))
end = dt.datetime.now()
print("operation took: %s" % str((end - start)))
print("Minimize result: %s" % str(min_result))

ERROR:analytics.batch:running eval sim 2 times with params: [0.2  0.05 0.05 0.01 0.05 0.01 0.05 0.01 0.2  0.02]
INFO:tutor.cogtutor_curriculum:Generated 2 units with with a total of 65 kcs


Bounds: [(0.2, 0.7), (0.05, 0.3), (0.05, 0.2), (0.01, 0.2), (0.05, 0.25), (0.01, 0.05), (0.05, 0.3), (0.01, 0.1), (0.2, 0.5), (0.02, 0.15)]


INFO:analytics.batch:Calculating stats for students
INFO:analytics.batch:Learner Transactions: (6979, 18)
INFO:tutor.cogtutor_curriculum:Generated 2 units with with a total of 67 kcs
INFO:analytics.batch:Calculating stats for students
INFO:analytics.batch:Learner Transactions: (6515, 18)
INFO:analytics.batch:Error:	{'Step attempts mean': 4.278574861914344, 'Step attempts std': 1.4290800098042258, 'Mean Pct Correct': 0.4996299641155759, 'Std Pct Correct': 0.07430020054573742, 'KC opportunity mean': 12.103950216450217, 'KC opportunity std': 13.07295801182925}
ERROR:analytics.batch:Run took 18.531175 seconds with mean err: 8.729303	 stdev err: 3.078057
ERROR:analytics.batch:running eval sim 2 times with params: [0.7  0.3  0.2  0.2  0.25 0.05 0.3  0.1  0.5  0.15]
INFO:tutor.cogtutor_curriculum:Generated 2 units with with a total of 77 kcs
INFO:analytics.batch:Calculating stats for students
INFO:analytics.batch:Learner Transactions: (5027, 18)
INFO:tutor.cogtutor_curriculum:Generated 2 unit

operation took: 4:52:24.537457
Minimize result:      fun: 2.7941340874457863
    funl: array([2.79413409, 2.93693716, 3.27508441, 3.35921697, 3.43049867,
       4.84995318])
 message: 'Optimization terminated successfully.'
    nfev: 1091
     nit: 2
   nlfev: 66
   nlhev: 0
   nljev: 6
 success: True
       x: array([0.7 , 0.05, 0.05, 0.01, 0.25, 0.05, 0.3 , 0.1 , 0.5 , 0.02])
      xl: array([[0.7 , 0.05, 0.05, 0.01, 0.25, 0.05, 0.3 , 0.1 , 0.5 , 0.02],
       [0.7 , 0.05, 0.05, 0.2 , 0.25, 0.01, 0.3 , 0.1 , 0.5 , 0.02],
       [0.7 , 0.05, 0.2 , 0.2 , 0.25, 0.05, 0.3 , 0.1 , 0.2 , 0.15],
       [0.7 , 0.3 , 0.2 , 0.01, 0.25, 0.01, 0.3 , 0.01, 0.2 , 0.02],
       [0.2 , 0.3 , 0.2 , 0.01, 0.25, 0.05, 0.3 , 0.1 , 0.2 , 0.15],
       [0.2 , 0.3 , 0.05, 0.2 , 0.25, 0.01, 0.05, 0.1 , 0.2 , 0.02]])


In [ ]:
print("operation took: %s" % str((end - start)))
print("Minimize result: %s" % str(min_result))

#### Optimial Result:


m_l0: 0.69975586 <br>
sd_l0: 0.0101416 <br>
m_l0_sd: 0.0101416 <br>
sd_l0_sd: 0.19990723 <br>
m_t: 0.1<br>
sd_t: 0.04998047  <br>
m_s: 0.3 <br>
sd_s: 0.01004395 <br>
m_g: 0.99960938 <br>
sd_g: 0.01009277 <br>

In [ ]:
# Pretty decent Optimization results:(From incomplete run)
target = {
    'Step attempts mean': 1,
    'Step attempts std': 0.4,
    'Mean Pct Correct': 0.8,
    'Std Pct Correct': 0.1,
    'KC opportunity mean': 7,
    'KC opportunity std': 3
}

Params = [0.7  0.3  0.05 0.01 0.25 0.05 0.3  0.1  0.5  0.15]
# Error:   {'Step attempts mean': 2.0029050705932154, 'Step attempts std': 1.4990586938742796, 
#           'Mean Pct Correct': 0.499478390461997, 'Std Pct Correct': 0.014261567469534915, 
#           'KC opportunity mean': 9.157894736842104, 'KC opportunity std': 2.902859417502669}

Params = [0.7  0.3  0.05 0.01 0.25 0.05 0.3  0.1  0.5  0.02]

Params = [0.7  0.3  0.05 0.01 0.25 0.05 0.3  0.1  0.2  0.02]
# Error:	{'Step attempts mean': 1.9355044699872286, 'Step attempts std': 1.5104306340938103, 
#            'Mean Pct Correct': 0.5298951048951049, 'Std Pct Correct': 0.1184280238420826, 
#            'KC opportunity mean': 7.0, 'KC opportunity std': 2.121320343559643}

Params = [0.7  0.3  0.2  0.2  0.25 0.05 0.3  0.01 0.2  0.15]
Error:	{'Step attempts mean': 2.183474991827395, 'Step attempts std': 2.034382015078027, 
         'Mean Pct Correct': 0.460415661404839, 'Std Pct Correct': 0.04730455167447055, 
         'KC opportunity mean': 6.5, 'KC opportunity std': 2.016563783383858}
    
    
# Optimum Result (2/2/21)
Params = [0.7 , 0.05, 0.05, 0.01, 0.25, 0.05, 0.3 , 0.1 , 0.5 , 0.02]
Other Close Optima
    [[0.7 , 0.05, 0.05, 0.01, 0.25, 0.05, 0.3 , 0.1 , 0.5 , 0.02],
    [0.7 , 0.05, 0.05, 0.2 , 0.25, 0.01, 0.3 , 0.1 , 0.5 , 0.02],
    [0.7 , 0.05, 0.2 , 0.2 , 0.25, 0.05, 0.3 , 0.1 , 0.2 , 0.15],
    [0.7 , 0.3 , 0.2 , 0.01, 0.25, 0.01, 0.3 , 0.01, 0.2 , 0.02],
    [0.2 , 0.3 , 0.2 , 0.01, 0.25, 0.05, 0.3 , 0.1 , 0.2 , 0.15],
    [0.2 , 0.3 , 0.05, 0.2 , 0.25, 0.01, 0.05, 0.1 , 0.2 , 0.02]]